In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('Restaurant_Reviews.csv')
data.head(10)

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
5,Now I am getting angry and I want my damn pho.,0
6,Honeslty it didn't taste THAT fresh.),0
7,The potatoes were like rubber and you could te...,0
8,The fries were great too.,1
9,A great touch.,1


## Data Exploration and Cleaning

In [3]:
cleaned_data = data.copy()


In [4]:
cleaned_data

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [5]:
cleaned_data['Review'].sample(1)

968    We've have gotten a much better service from t...
Name: Review, dtype: object

In [6]:
cleaned_data['Review'].iloc[643]

"Maybe if they weren't cold they would have been somewhat edible."

In [7]:
import re

transformed_review = []
for review in  cleaned_data['Review']:
    transformed_review.append(re.sub('[^a-zA-Z]', ' ', review))


In [8]:
from sklearn.feature_extraction import text

import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer(mode='NLTK_EXTENSIONS')
stop_words = [word for word in stopwords.words('english') if word != 'not']

In [9]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    # Tokenizing
    tokens = word_tokenize(text)
    # Removing stopwords
    tokens = [token for token in tokens if token not in stop_words]
    # Stemming
    stems = stem_tokens(tokens, stemmer)
    return stems

In [10]:
cleaned_data['Review'] = transformed_review

In [11]:
vectorizer = text.CountVectorizer(tokenizer=tokenize)

In [12]:
vectorizer.fit_transform(cleaned_data['Review']).toarray()

C:\Users\Administrator\anaconda3\envs\Tensor_Flow\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
# function to standardize text

# remove all punctuations
# remove all stop words
# stem words